<a href="https://colab.research.google.com/github/mars241/Machine-Learning/blob/main/Corrig%C3%A9_FAFI_Moncef_2_2_ML_Classification_KNN_distances_with_Pokemons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Importation des librairies et initialisation du DataFrame

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier

df_pokemon = pd.read_csv('https://raw.githubusercontent.com/murpi/wilddata/master/pokemon.csv', sep=',')

In [ ]:
df_pokemon.info()
# On remarque la colonne Type 2 qui a seulement 414 valeures sur 800
# On choisit de l'ignorer

df_pokemon = df_pokemon.drop(columns=['Type 2'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        800 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   HP          800 non-null    int64 
 5   Attack      800 non-null    int64 
 6   Defense     800 non-null    int64 
 7   Sp. Atk     800 non-null    int64 
 8   Sp. Def     800 non-null    int64 
 9   Speed       800 non-null    int64 
 10  Generation  800 non-null    int64 
 11  Legendary   800 non-null    bool  
dtypes: bool(1), int64(8), object(3)
memory usage: 69.7+ KB


# 2. On s'intéresse dans le détail aux colonnes

## Colonne `#` :
Valeur numérique, mais ne doit surtout pas être pris en compte dans l'algo. C'est simplement un identifiant, la proximité entre les pokémons ne dépend absolument pas de cette valeur

## Colonne `Name` :
Dtype object, un texte indiquant le nom du Pokémon, sert simplement à nommer et identifier le pokémon.

## Colonne `Type 1` :
Dtype object, un texte pour donner le type du Pokémon.  
La valeur du champ est déterminante dans le choix d'un pokémon, on a besoin de l'intégrer au modèle mais un traitement est nécessaire.  
Avec `nunique()` sur la colonne on trouve 18 valeurs. Plutôt que de garder le type object, on peut au choix utiliser `factorize()` ou `get_dummies()` pour avoir des nombres.
`factorize()` va classifier les 18 types en 18 valeurs numériques, mais la proximité de deux types (2 et 3, ou 4 et 18) va impacter notre modèle qui va attribuer un poids différent.  
On va partir sur `get_dummies()`, même si ça va rajouter 18 colonnes x 800 lignes, et on comparera après coup avec un factorize au besoin.

## Colonne `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed` :
Colonnes en int64, qui donnent respectivement les nombre de points de vie, points d'attaque / défense, attaque et défense spéciale, vitesse d'attaque d'un pokémon.
Toutes utiles au modèle, et déterminent les performances, et donc le choix d'un pokémon. Aucun traitement supplémentaire sur ces colonnes.

## Colonne `Generation` :
Colonne en int64, qui identifie la génération du pokémon entre la 1ère et la 6ème.
C'est une valeur numérique, mais elle n'a aucune incidence sur la performance d'un pokémon, elle sert simplement à les classifier.
On peut donc l'ignorer

## Colonne `Legendary` :
Colonne qui indique par un booléen si le pokémon est légendaire ou non.   
Globalement, un pokémon légendaire a des caractéristiques boostées donc plus puissant qu'un pokémon classique.  
On doit rechercher uniquement des pokémon classiques et on pourrait être tenté d'ignorer cette colonne.  
On choisit de la traiter car elle a un impact sur les autres valeures.  
Elle détermine en partie la classification des pokémon et donc leur distance dans notre modèle.  
Pour le résultat final, on ignorera simplement les voisins proches qui sont des pokémons légendaires.
On la garde sous forme numérique avec un `factorize()` :


In [ ]:
# On éclate la colonne `Type 1` avec `get_dummies()` et on convertit la colonne `Legendary`
df_pokemon = pd.concat([df_pokemon,df_pokemon['Type 1'].str.get_dummies()], axis = 1)
df_pokemon['Legendary'] = df_pokemon['Legendary'].factorize()[0]

# 3. Le DataFrame est prêt, on peut commencer le traitement du problème suivant :
Remplacer les pokémons légendaires par les pokémons classiques les plus proches dans l'équipe du champion :  
Mewtwo, Lugia, Rayquaza, Giratina, Dialga, et Palkia.

In [ ]:
equipe_originale = ['Mewtwo', 'Lugia', 'Rayquaza', 'Giratina', 'Dialga', 'Palkia']

def cherche_legendaires(une_liste):

  dict_legend = {}
  for i in  une_liste:
    dict_legend[i] = df_pokemon.loc[df_pokemon['Name'].str.contains(i), 'Legendary'].iloc[0]
  return dict_legend


cherche_legendaires(equipe_originale)
# Les 6 pokémons sont des pokémons légendaires, il faut créer une nouvelle équipe complète

{'Mewtwo': 1,
 'Lugia': 1,
 'Rayquaza': 1,
 'Giratina': 1,
 'Dialga': 1,
 'Palkia': 1}

In [ ]:
from sklearn.neighbors import NearestNeighbors

# On définit les colonnes qui vont servir au calcul des voisins, ce sont les colonnes que l'on doit renseigner avec kneighbors également
filtre_colonnes = ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def',
       'Speed', 'Legendary', 'Bug', 'Dark', 'Dragon', 'Electric',
       'Fairy', 'Fighting', 'Fire', 'Flying', 'Ghost', 'Grass', 'Ground',
       'Ice', 'Normal', 'Poison', 'Psychic', 'Rock', 'Steel', 'Water']

def init_distanceKNN(un_nombre_de_voisins, une_liste_colonnes_pour_X):
  X = df_pokemon[une_liste_colonnes_pour_X]
  X = X.values
  return NearestNeighbors(n_neighbors=un_nombre_de_voisins).fit(X)

def cherche_stats_equipe(une_liste_de_pokemon):
  dict_stats_equipe = {}
  # On alimente un dictionnaire avec les pokemon de l'équipe originale, et leurs stats en suivant la liste de colonnes filtre_colonnes
  for i in  une_liste_de_pokemon:
    dict_stats_equipe[i] = df_pokemon.loc[df_pokemon['Name'].str.contains(i), filtre_colonnes].iloc[0].tolist()
  return dict_stats_equipe


# On prends le dataframe réduit (sans les colonne '#', 'Name', 'Type 1', et 'Generation') pour chercher les voisins les plus proches
# On choisit n=10 afin de ne pas avoir que des légendaires dans le résultat des proches voisins plus bas
distanceKNN = init_distanceKNN(10,filtre_colonnes)
dict_stats_equipe_originale = cherche_stats_equipe(equipe_originale)

#Affichage pour contrôle
print(distanceKNN,'\n',dict_stats_equipe_originale)

NearestNeighbors(n_neighbors=10) 
 {'Mewtwo': [106, 110, 90, 154, 90, 130, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], 'Lugia': [106, 90, 130, 90, 154, 110, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], 'Rayquaza': [105, 150, 90, 150, 90, 95, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'Giratina': [150, 100, 120, 100, 120, 90, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'Dialga': [100, 120, 120, 150, 100, 90, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], 'Palkia': [90, 120, 100, 150, 120, 100, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]}


# 4. On fais le test avec Mewtwo, on généralisera par la suite

In [ ]:
# On teste les proches voisins de Mewtwo :
distanceKNN.kneighbors([dict_stats_equipe_originale['Mewtwo']])

# Le 1er array renvoie la liste des distances, 0 = lui-même, 41 à 47, les 9 voisins suivants
# Le 2ème array renvoie la liste des numéros des pokemons, 162 = Mewtwo, 551 = Shaymin Sky Forme, 248 = Mega Houndoom etc...

(array([[ 0.        , 41.25530269, 42.24926035, 43.06971093, 45.21061822,
         45.54119015, 45.65084884, 45.70557953, 46.66904756, 47.68647607]]),
 array([[162, 551, 248, 275, 712,  23, 549, 696, 705, 541]]))

In [ ]:
# On doit trouver le voisin le plus proche, qui ne soit pas un légendaire

# On récupère les 2 array pour ses 10 voisins
voisins_n_10 = distanceKNN.kneighbors([dict_stats_equipe_originale['Mewtwo']])

for i in range(1,10):
  remplacant = df_pokemon[df_pokemon['#'] == voisins_n_10[1][0][i] +1]['Legendary'] == 0
  if remplacant.bool() :
    break
""" ================================   EXPLICATION DU FOR   ================================================
remplacant : une variable pour chercher le pokemon candidat qui remplacera Mewtwo dans l'exemple

Algo :
voisins_n_10[1][0][i] : On parcours le 2ème array [1][0], pour tous les [i], donc pour les voisins de 1 à 10 => liste les numéros '#' des pokémons voisins
On à n = 10 voisins. Mais on commence la boucle à 1 pour ignorer Mewtwo
voisins_n_10[1][0][i] +1 : Il y a un décalage de 1 entre l'index qui commence à 0, et le numéro des pokémons qui commence à 1, donc on le répercute ici pour identifier le bon
df_pokemon[df_pokemon['#'] == voisins_n_10[1][0][i] +1] : On applique le dataframe pour récupérer toutes les infos du pokemon qui match avec le numéro retourné par voisins_n_10
Enfin on s'intéresse juste à la colonne Legendary qui doit etre égale à 0.
Le résultat de ce test logique est un booléen stocké dans la variable remplacant
Dès qu'on trouve une valeur à vraie, c'est qu'on a le plus proche voisin non légendaire, on peut donc sortir de la boucle
====================================   FIN EXPLICATION   ====================================================
"""

# Affichage pour contrôle :
print('Variable remplacant : \n',remplacant,'\n\nNom du pokémon remplaçant : \n', df_pokemon[df_pokemon['#']==remplacant.index[0]+1]['Name'],'\n')

# On trouve donc que Mewtwo doit être remplacé par le 248 qui est 'Mega Houndoom'

Variable remplacant : 
 248    True
Name: Legendary, dtype: bool 

Nom du pokémon remplaçant : 
 248    Mega Houndoom
Name: Name, dtype: object 



# 5. Généraliser le traitement pour toute l'équipe et retourner une liste de 6 pokémons similaire à equipe_originale

## 5.1 Création de fonctions pour le bloc 3. à l'initialisation de *distanceKNN*

## 5.2 Dans le bloc 4. au lieu de renseigner Mewtwo, faire le traitement avec toute l'équipe originale


In [ ]:
"""
Bloc avec beaucoup de commentaires pour garder une trace du raisonnement au besoin d'y replonger
C'est exactement le même code dans le bloc qui suit, passer directement à la suite au besoin
"""

# On ré initialise distanceKNN, on garde n=10 pour éviter les légendaires ou les doublons
# Variables locales :
nb_voisins=10
distanceKNN = init_distanceKNN(nb_voisins,filtre_colonnes)
liste_remplacants=[]

# Decommenter les print pour checker la valeurs des variables dans les deux boucles, et vérifier le if

# On parcourt chaque pokemon de la liste originale
# On fait la recherche des voisins de chaque pokemon_source, pour trouver son pokemon_remplacant
for pokemon in equipe_originale:
  #print('\n||||On est dans la boucle pokemon dans equipe_originale, au tour de',pokemon)
  stats_pokemon_source = cherche_stats_equipe(equipe_originale)[pokemon]
  voisins_pokemon_source = distanceKNN.kneighbors([stats_pokemon_source])
  for voisins in range(1,nb_voisins):
    #print('\n**On est dans la boucle voisins, pour le numero :',voisins)
    #print(df_pokemon[df_pokemon['#'] == voisins_pokemon_source[1][0][voisins] +1]['Name'])
    pokemon_remplacant = df_pokemon[df_pokemon['#'] == voisins_pokemon_source[1][0][voisins] +1]['Legendary'] == 0

    """
    NOTE : On a parcouru l'EQUIPE
              Puis on a parcouru les VOISINS
                    Maintenant on test 2 choses :
                        *  Si le test pokemon_remplacant renvoie vrai (voisin le plus proche non légendaire)
                        *  Si ce pokémon n'a pas déjà été ajouté dans la liste.
                          On cherche sa valeur numérique stocké dans la série pokemon_remplacant via index[0] et on utilise .count == 0 pour vérifier son absence dans la liste_remplacants.
    """
    if pokemon_remplacant.bool() and liste_remplacants.count(pokemon_remplacant.index[0]) == 0:
      # print("Le test bool est vrai pour les param : \nPokemon_source :",pokemon,
      #       '\nVoisin numéro :',voisins,
      #       '\nNumero du pokemon remplaçant :',pokemon_remplacant.index[0],
      #       '\nEtat de la liste des remplacants avant ajout : ',ls)
      break
  liste_remplacants.append(pokemon_remplacant.index[0])
  #print('Etat de la liste après ajout : ', liste_remplacants)

print(liste_remplacants)

[248, 546, 279, 271, 12, 8]


## 5.3 On remets ça au propre avec une fonction qui prends en paramètre une équipe, et renvoie une liste de remplaçants

In [ ]:
nb_voisins=10
distanceKNN = init_distanceKNN(nb_voisins,filtre_colonnes)

def remplace_equipe(une_liste_de_pokemon):
  liste_remplacants=[]

  for pokemon in une_liste_de_pokemon:
    stats_pokemon_source = cherche_stats_equipe(une_liste_de_pokemon)[pokemon]
    voisins_pokemon_source = distanceKNN.kneighbors([stats_pokemon_source])
    for voisins in range(1,nb_voisins):
      pokemon_remplacant = df_pokemon[df_pokemon['#'] == voisins_pokemon_source[1][0][voisins] +1]['Legendary'] == 0
      if pokemon_remplacant.bool() and liste_remplacants.count(pokemon_remplacant.index[0]) == 0:
        break
    liste_remplacants.append(pokemon_remplacant.index[0])
  return liste_remplacants


## 5.4 Plus de lisibilité : on convertit les numéros en noms de Pokémons

In [ ]:
def convertit_numero_pokemon(liste_de_numero):
  ls = []
  for i in liste_de_numero:
    ls.append(df_pokemon.iloc[i]['Name'])
  return ls

# 6. Enfin, on affiche la fameuse équipe

In [ ]:
equipe_remplacee = convertit_numero_pokemon(remplace_equipe(equipe_originale))
print("La liste des 6 pokemon qui remplaceront au mieux :\n", equipe_originale,"\n\nsont : \n",equipe_remplacee)
# On vérifie qu'aucun des pokemon ne soit légendaire
print("\n\nOn affiche un dictionnaire avec en valeur le bool 'Legendary' pour s'assurer que la condition legendaires est respectée malgré la présence de Celebi et Cresselia : \n",
      cherche_legendaires(equipe_remplacee))

La liste des 6 pokemon qui remplaceront au mieux :
 ['Mewtwo', 'Lugia', 'Rayquaza', 'Giratina', 'Dialga', 'Palkia'] 

sont : 
 ['Mega Houndoom', 'Cresselia', 'Mega Blaziken', 'Celebi', 'Mega Blastoise', 'Mega Charizard Y']


On affiche un dictionnaire avec en valeur le bool 'Legendary' pour s'assurer que la condition legendaires est respectée malgré la présence de Celebi et Cresselia : 
 {'Mega Houndoom': 0, 'Cresselia': 0, 'Mega Blaziken': 0, 'Celebi': 0, 'Mega Blastoise': 0, 'Mega Charizard Y': 0}


In [ ]:
#On teste une autre équipe
equipe_de_sacha = ['Pikachu', 'Lucario', 'Dragonite','Gengar','Blastoise']
print(convertit_numero_pokemon(remplace_equipe(equipe_de_sacha)))

['Poliwag', 'Blaziken', 'Tyranitar', 'Espeon', 'Meganium']


In [ ]:
equipe = ['Salamence','Blaziken','Lugia','Suicune','Groudon','Metagross']
convertit_numero_pokemon(remplace_equipe(equipe))

['Mega Glalie', 'Lucario', 'Cresselia', 'Blastoise', 'Metagross', 'Tyranitar']